Import tools to help communicate with Fred API
Requests helps us send messages over the internet
Optional lets us say that sometimes a piece of information might not be there.

In [27]:
import requests
from typing import Optional

### Creates a blueprint for our FRED API tool.

In [28]:
class FredPyAPI:
    # main web address our application will talk to
    BASE_URL = "https://api.stlouisfed.org/fred/"

### This is how we create our robot (Initialize).  We can give it a token (a special password) 
### right away, or we can add it later.

In [ ]:
def __init__(self, token: Optional[str] = None):
        self.token = token

### This is a way to give our robot the special password after we've created it.

In [29]:
    def set_token(self, token: str) -> None:
        self.token = token

### This is a private helper function (that's why it starts with _). It's like the robot's 
### internal communication system.

In [30]:
    def _make_request(self, endpoint: str, params: dict) -> dict:
        # This checks if the robot has its password (api_key). If not, it stops and tells us to set the password.
        if not self.token:
           raise ValueError("API token is not set. Use set_token() method to set it.")
           # These lines add the password and ask for the response in a specific format (JSON).
           params['api_key'] = self.token
           params['file_type'] = 'json'
    

### This sends the request to FRED, checks for errors, and returns the data.

In [ ]:
         response = requests.get(f"{self.BASE_URL}{endpoint}", params=params)
         response.raise_for_status()
         return response.json()

### This function gets data about a specific economic series (like GDP).

In [ ]:
    def get_series_data(self, series_id: str) -> dict:
        return self._make_request("series", {"series_id": series_id})

### This gets historical data for a series, with optional start and end dates.

In [ ]:
    def get_historical_data(self, series_id: str, observation_start: Optional[str] = None, observation_end: Optional[str] = None) -> dict:
        params = {"series_id": series_id}
        if observation_start:
            params["observation_start"] = observation_start
        if observation_end:
            params["observation_end"] = observation_end
        return self._make_request("series/observations", params)

### This gets information about categories of economic data.

In [ ]:
    def get_categories(self, category_id: int = 0) -> dict:
        return self._make_request("category", {"category_id": category_id})

### This gets information about when new economic data is released.

In [ ]:
    def get_releases(self, realtime_start: Optional[str] = None, realtime_end: Optional[str] = None) -> dict:
        params = {}
        if realtime_start:
            params["realtime_start"] = realtime_start
        if realtime_end:
            params["realtime_end"] = realtime_end
        return self._make_request("releases", params)

### The comments at the end show examples of how to use this tool:

In [ ]:
# Usage example:
# fred_api = FredPyAPI()
# fred_api.set_token("your_api_key_here")
# series_data = fred_api.get_series_data("GDP")
# historical_data = fred_api.get_historical_data("GDP", observation_start="2020-01-01", observation_end="2023-12-31")
# categories = fred_api.get_categories()
# releases = fred_api.get_releases(realtime_start="2023-01-01")